# Install all packages

In [217]:
!pip install requests
!pip install html5lib
!pip install bs4
from bs4 import BeautifulSoup
import requests

import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
%matplotlib inline
from sklearn import linear_model
from sklearn import metrics

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# Scrape datasets - Toronto

In [206]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = requests.get(URL)
soup = BeautifulSoup(html.content, 'html5lib')

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df_toronto =pd.DataFrame(table_contents)
df_toronto['Borough']=df_toronto['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

lat_lon = pd.read_csv(r"C:\Users\siqm\Downloads\Geospatial_Coordinates.csv")
lat_lon = lat_lon.rename(columns = {'Postal Code': 'PostalCode'})

df_toronto = pd.merge(df_toronto, lat_lon, on = 'PostalCode')
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


# Toronto Census Data for Income, Age, Languages and Population

In [207]:
census_data = pd.read_csv(r"C:\Users\siqm\Downloads\toronto_data.csv")
census_data.dropna(how='all', axis=1, inplace=True)

df_toronto = pd.merge(df_toronto, census_data, on = 'Neighborhood')
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Pop_density,Age_0_14,Age_15_24,Age_25_54,Age_55_64,...,Inc_45000_49999,Inc_50000_59999,Inc_60000_69999,Inc_70000_79999,Inc_80000_89999,Inc_90000_99000,Inc_100000_199999,Inc_200000,Inc_0_50000,Inc_50000_over
0,M3A,North York,Parkwoods,43.753259,-79.329656,4691.0,0.164,0.131,0.415,0.120,...,0.047,0.090,0.077,0.066,0.059,0.049,0.285,0.067,0.336,1.029
1,M4A,North York,Victoria Village,43.725882,-79.315572,3710.0,0.146,0.112,0.397,0.126,...,0.057,0.086,0.077,0.066,0.052,0.039,0.195,0.032,0.410,0.957
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,16880.0,0.150,0.163,0.513,0.101,...,0.042,0.080,0.073,0.051,0.044,0.035,0.150,0.030,0.484,0.947
3,M6C,York,Humewood-Cedarvale,43.693781,-79.428191,7682.0,0.148,0.110,0.478,0.113,...,0.046,0.078,0.065,0.055,0.042,0.037,0.311,0.142,0.351,1.081
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,6058.0,0.175,0.095,0.439,0.144,...,0.031,0.054,0.048,0.047,0.046,0.043,0.503,0.240,0.232,1.213


# Get all venues from Foursquare API

In [208]:
CLIENT_ID = 'CVSUDURB54AQFHZE55RFYO3QT5Z4DZCFYZMNWWYNCCR2KKRZ' # your Foursquare ID
CLIENT_SECRET = 'LUSTZ5KPZFXKXXD44FLGB1GORLIBYUOOX5JV5B2JFME1ZMCH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [209]:
# Find number of venues for each neighbourhood
toronto_data = {}
toronto_venues = {}
for name in df_toronto["Borough"].unique():   
    toronto_data[name] = df_toronto[df_toronto['Borough'].str.contains(name)].reset_index(drop=True)

    toronto_venues[name] = getNearbyVenues(names=toronto_data[name]['Neighborhood'],
                                   latitudes=toronto_data[name]['Latitude'],
                                   longitudes=toronto_data[name]['Longitude']
                                  )

Parkwoods
Victoria Village
Hillcrest Village
Bayview Village
Humber Summit
Willowdale West
Regent Park, Harbourfront
Parkwoods
Victoria Village
Humewood-Cedarvale
Hillcrest Village
Thorncliffe Park
Bayview Village
Humber Summit
Weston
Willowdale West
The Beaches
Woburn
Scarborough Village
Thorncliffe Park


# Limiting to Just Artistic Venues

In [210]:
toronto_onehot = {}
toronto_grouped = {}
col = {}

for name in df_toronto["Borough"].unique(): 
    # one hot encoding
    toronto_onehot[name] = pd.get_dummies(toronto_venues[name][['Venue Category']], prefix="", prefix_sep="")

    # add neighborhood column back to dataframe
    toronto_onehot[name]['Neighborhood'] = toronto_venues[name]['Neighborhood'] 

    toronto_grouped[name] = toronto_onehot[name].groupby('Neighborhood').mean().reset_index()
    
    # Merge raw data with venues data
    toronto_grouped[name] = pd.merge(toronto_data[name], toronto_grouped[name], on = 'Neighborhood')
    
    # Keep artistic venues only
    col[name] = list(toronto_grouped[name].columns.values)
    matchers = ['Restaurant','Neighborhood','Lan','Inc','Pop','Age']
    #matchers = ['Art','Neighborhood','Record','Entertainment','Dance','Jazz','Venue','Sculpture',\
                #'Opera','Theater','Speakeasy','Book','Lan','Inc','Pop','Age'] # Not enough art venues
    matching[name] = [s for s in col[name] if any(xs in s for xs in matchers)]
    
    toronto_grouped[name] = toronto_grouped[name][matching[name]]
    
toronto_grouped['North York'].head()

,Neighborhood,Pop_density,Age_0_14,Age_15_24,Age_25_54,Age_55_64,Age_65_84,Age_85,Age_young,Age_old,...,Inc_90000_99000,Inc_100000_199999,Inc_200000,Inc_0_50000,Inc_50000_over,Chinese Restaurant,Fast Food Restaurant,Japanese Restaurant,Mediterranean Restaurant,Portuguese Restaurant
0,Parkwoods,4691.0,0.164,0.131,0.415,0.120,0.148,0.022,0.711,0.289,...,0.049,0.285,0.067,0.336,1.029,0.00,0.333333,0.00,0.000000,0.00
1,Victoria Village,3710.0,0.146,0.112,0.397,0.126,0.188,0.032,0.654,0.346,...,0.039,0.195,0.032,0.410,0.957,0.00,0.000000,0.00,0.000000,0.25
2,Hillcrest Village,3148.0,0.121,0.124,0.345,0.129,0.245,0.036,0.590,0.410,...,0.048,0.346,0.075,0.292,0.986,0.00,0.166667,0.00,0.166667,0.00
3,Bayview Village,4195.0,0.110,0.114,0.469,0.115,0.164,0.028,0.692,0.308,...,0.052,0.316,0.090,0.340,1.055,0.25,0.000000,0.25,0.000000,0.00
4,Humber Summit,1570.0,0.170,0.136,0.383,0.104,0.188,0.020,0.688,0.312,...,0.047,0.249,0.030,0.376,1.004,0.00,0.000000,0.00,0.000000,0.00


# Append each borough's dataset to make one Toronto dataset

In [211]:
boroughs = list(toronto_grouped.keys())
first = boroughs[0]
print(boroughs)
boroughs.pop(0)

df_combined = toronto_grouped[first]

for b in boroughs:
    df_combined = df_combined.append(toronto_grouped[b])


['North York', 'Downtown Toronto', 'York', 'East Toronto', 'Scarborough', 'East York']


# Collapse into the number of restaurants per neighbourhood

In [212]:
cols_restaurants = [col for col in df_combined if col.endswith('aurant')]
df_combined['Sum of Restaurants'] = df_combined[cols_restaurants].sum(axis=1)

df_combined.drop(cols_restaurants, axis=1, inplace = True) # Drop extra columns
df_combined.head()

,Neighborhood,Pop_density,Age_0_14,Age_15_24,Age_25_54,Age_55_64,Age_65_84,Age_85,Age_young,Age_old,...,Inc_50000_59999,Inc_60000_69999,Inc_70000_79999,Inc_80000_89999,Inc_90000_99000,Inc_100000_199999,Inc_200000,Inc_0_50000,Inc_50000_over,Sum of Restaurants
0,Parkwoods,4691.0,0.164,0.131,0.415,0.120,0.148,0.022,0.711,0.289,...,0.090,0.077,0.066,0.059,0.049,0.285,0.067,0.336,1.029,0.333333
1,Victoria Village,3710.0,0.146,0.112,0.397,0.126,0.188,0.032,0.654,0.346,...,0.086,0.077,0.066,0.052,0.039,0.195,0.032,0.410,0.957,0.250000
2,Hillcrest Village,3148.0,0.121,0.124,0.345,0.129,0.245,0.036,0.590,0.410,...,0.059,0.055,0.063,0.048,0.048,0.346,0.075,0.292,0.986,0.333333
3,Bayview Village,4195.0,0.110,0.114,0.469,0.115,0.164,0.028,0.692,0.308,...,0.068,0.069,0.065,0.055,0.052,0.316,0.090,0.340,1.055,0.500000
4,Humber Summit,1570.0,0.170,0.136,0.383,0.104,0.188,0.020,0.688,0.312,...,0.085,0.083,0.069,0.065,0.047,0.249,0.030,0.376,1.004,0.000000


# Regression Analysis

In [213]:
list(df_combined.columns)

['Neighborhood',
 'Pop_density',
 'Age_0_14',
 'Age_15_24',
 'Age_25_54',
 'Age_55_64',
 'Age_65_84',
 'Age_85',
 'Age_young',
 'Age_old',
 'Lan_eng',
 'Lan_fren',
 'Lan_en_fren',
 'Lan_neither',
 'Lang_native',
 'Lang_foreign',
 'Inc_0_4999',
 'Inc_5000_9999',
 'Inc_10000_19999',
 'Inc_20000_24999',
 'Inc_25000_29999',
 'Inc_30000_34999',
 'Inc_35000_39999',
 'Inc_40000_44999',
 'Inc_45000_49999',
 'Inc_50000_59999',
 'Inc_60000_69999',
 'Inc_70000_79999',
 'Inc_80000_89999',
 'Inc_90000_99000',
 'Inc_100000_199999',
 'Inc_200000',
 'Inc_0_50000',
 'Inc_50000_over',
 'Sum of Restaurants']

In [219]:
# Regress number of restaurants on population density, age dummies, language dummies and income dummies
regr = linear_model.LinearRegression()
x = np.asanyarray(df_combined[['Pop_density','Age_young','Age_old','Lang_native','Lang_foreign','Inc_0_50000', 'Inc_50000_over']])
y = np.asanyarray(df_combined[['Sum of Restaurants']])

regr.fit (x, y)

# The coefficients
print ('Coefficients: ', regr.coef_)

# Model Evaluation
y_hat= regr.predict(df_combined[['Pop_density','Age_young','Age_old','Lang_native','Lang_foreign','Inc_0_50000', 'Inc_50000_over']])
r2 = metrics.r2_score(y, y_hat)

# Explained variance score: 1 is perfect prediction
print('R-squared: %.2f' % r2)
print('Variance score: %.2f' % regr.score(x, y))


Coefficients:  [[-1.00930659e-05  1.17054479e+00 -1.17054479e+00  3.71979700e+01
   3.58786035e+01 -4.40701671e+00 -3.68054996e+00]]
R-squared: 0.27
Variance score: 0.27
